In [1]:
from _util import *
import _RL.FQE as FQE_module
import _RL.FQI as FQI
import _RL.my_gym as my_gym
import _TRIPLE as TRIPLE
import _analyze as analyze
import _RL.DQN as DQN
import _RL.my_gym as my_gym
import _cartpole as cartpole
reload(cartpole)
reload(my_gym)

reload(TRIPLE)
reload(analyze)

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
tf.keras.backend.set_floatx('float32')


## N = 10, T = 300, tau = 0.3

In [ ]:
###################################################################################################################
unique_setting = "cart_w_Q"
N = 10

n_gpu = 8

rep = 200
tau = 0.3
T = 300
data_exist = 0
env = cartpole.CartPoleEnv(e_max = 1000, std = 0.01)

########################################################################
gamma = .98
init_Q_ratio = 0.99
incomplete_ratio = 20
FQE_paras = {"verbose" : 1, "test_freq" : 50
             , "eps" : 0.0001, "max_iter" : 200, "init_Q_ratio" : init_Q_ratio
             , "max_depth" : 20, "n_estimators" : 1000, "min_samples_leaf" : 10
             ,  "use_RF" : 1}
################################################################################################################################################ 
###################################################### Default Settings ######################################################
################################################################################################################################################
hyperparameters = [N, T, gamma, tau, init_Q_ratio, incomplete_ratio]
################## estimate the target policy [fixed number of iterations?] ##################
setting = EST()[7:9] + EST()[10:13]+ "_"  + unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) + "_" + "initQ" + str(init_Q_ratio) + "_"  + "tau" + "_" + str(tau)
printR("setting: {}".format(setting))
################################################################################################################## 
weight_path = "target_policies/" + "cartpole" + "g" + str(0.99)
tp_path = weight_path + "/iter" + str(20000)
##################################################################################################################
##################################################################################################################
###################
if data_exist:
    data = load("data/" + Q_file)
    trajs_train_resp = data['trajs_train_resp']
    V_true = data['V_true']
    init_S = data['init_S']
    print("V_true = {:.3f}".format(V_true))
else:
    pi1 = DQN.DQN_gym(num_states = 4, num_actions = 2
                      , hidden_units = [256, 256], gamma = gamma
                      , gpu_number = 0)
    pi1.model.load_weights(tp_path)
    #########################################################
    pi_behav = my_gym.softmax_policy(pi1, tau)

    gym_eval = my_gym.GymEval(random_pi = True)
    trajs = gym_eval.simu_trajs_para(pi_behav, rep = 10000, burn_in = 500)
    init_S = gym_eval.get_init_S_from_trajs(trajs, n_init = 1000) # use in estimation
    # 
    init_S_4_eval = gym_eval.get_init_S_from_trajs(trajs, n_init = 10000)
    V_true = gym_eval.eval_policy(pi = pi1, gamma = gamma, init_states = init_S_4_eval, rep = len(init_S_4_eval))
    #
    trajs = gym_eval.simu_trajs_para(pi_behav, rep = N * rep * 2, burn_in = 500)
    trajs_train_resp = [trajs[(i * N):((i + 1) * N)] for i in range(rep)]
    data = {"trajs_train_resp" : trajs_train_resp
           , "V_true" : V_true
           , "init_S" : init_S}
    dump(data, "data/" + Q_file)

    
##################################################################################################################
##################################################################################################################
ray.shutdown()
a = now()
rec = analyze.recorder()
rec.hyper = hyperparameters
for i in range(rep // n_gpu):
    traj_this_rep = [trajs_train_resp[i] for i in range(n_gpu * i, n_gpu * (i + 1))]
    @ray.remote(num_gpus=1)
    def one_seed1(seed):
        tf.keras.backend.set_floatx('float32')
        gpu_idx = seed % n_gpu
        pi1 = DQN.DQN_gym(num_states = 4, num_actions = 2
                  , hidden_units = [256, 256], gamma = gamma
                  , gpu_number = gpu_idx)
        pi1.model.load_weights(tp_path)
        trajs_train = traj_this_rep[gpu_idx]
        ############################ TRIPLE to estimate values ############################
        are = TRIPLE.ARE(trajs_train, pi = pi1, gamma = gamma, gpu_number = gpu_idx, incomplete_ratio = incomplete_ratio, L = 2
                        , sepe_A = True)
        are.init_S = init_S 
        are.est_Q(**FQE_paras)
#         are.load_Q(Q_values = Qs[seed])
        are.est_w(h_dims = 512, batch_size = 256, lr = 0.001, print_freq = 300, tolerance = 10, max_iter = 500, rep_loss = 5)
        are.est_IS()
        are.est_double_robust()
        #######
        are.est_cond_w(h_dims = 512, batch_size = 32, lr = 0.0001, print_freq = 300, tolerance = 10, max_iter = 500, rep_loss = 5)
        are.est_triply_robust()  
        are.est_quad_robust()
        #######
        # analyze.recorder().print_one_seed(V_true, are)
        return [are.raw_Qs, are.IS_V, are.DR_V, are.TR_V, are.QR_V]
    #################################
    ray.init()
    futures = [one_seed1.remote(j) for j in range(n_gpu * i, n_gpu * (i + 1))]
    res = ray.get(futures)
    #################################
    for j in range(n_gpu):
        rec.update(V_true, are_details = res[j])
    _ = rec.analyze()
    #################################
    printR("time cost for {}-th rep = {:.1f} mins".format(i, (now() - a) / 60))
    rec.save("res/" + setting)
    ray.shutdown()
    #################################